## 1.2 신경망의 추론
### 1.2.1 신경망 추론 전체 그림
- 신경망은 '함수'
- 함수는 입력된 무언가를 출력으로 변환하는 변환기


e.g. 2차원 데이터를 입력하여 3차원 데이터를 출력하는 함수
- 입력층(input layer): 뉴런 2개
- 출력층(output layer): 뉴런 3개
- 은닉층(hidden layer): 적당한 수의 뉴런 배치 

- 이 때, 입력에서부터 출력 방향으로 각 측의 뉴런들은 화살표로 연결된다. 
    - 또한 이 화살표에는 가중치가 존재한다.
    - 각 가중치와 뉴런의 값을 각각 곱해서 그 합이 다음 뉴런의 입력으로 쓰이게 된다.
        - 정확히는 그 합에 활성화 함수를 적용한 값이 다음 뉴런의 입력이 된다.)
    - 이 때, 각 층에서는 이전 뉴런 값에 영향을 받지 않는 '정수'도 더해진다. (= 편향(bias))
- 인접하는 모든 층의 뉴런과 연결되어 있다면, 이를 완전 연결 계층(fully connected layer)라고 한다.


- 신경망의 추론이나 학습에서는 다수의 샘플 데이터(*미니 배치)를 한꺼번에 처리한다.
- 이렇게 하려면 행렬 x의 행 각각에 샘플 데이터를 하나씩 저장해야한다.
- e.g. N개의 샘플 데이터를 미니 배치로 한꺼번에 처리하는 경우 (편향 생략)
    - x W = h
    - (`N X 2`) (`2 X 4`) = `N x 4`
    - N개의 샘플 데이터가 한꺼번에 완전연결계층에 의해 변환되고, 은닉층에는 N개 분의 뉴런이 함께 계산 된다.

In [1]:
import numpy as np

W1 = np.random.randn(2, 4) # 가중치
b1 = np.random.randn(4)    # 편향
x = np.random.randn(10, 2) # 입력, 10개의 샘플 데이터
h = np.matmul(x, W1) + b1

- 10개의 샘플 데이터.
- x의 첫번째 차원이 각 샘플 데이터에 해당
    - e.g. x[0]은 0번째 입력데이터, x[1]은 1번째 입력 데이터,...
    - e.g. h[0]은 0번째 데이터의 은닉층 뉴런, h[1]은 1번째  데이터의 은닉층 뉴런
- h에서 더해지는 편행 b1은 브로드캐스트된다. b1은 (4,)의 형상이지만, (10, 4)로 확장되어 더해진다.

- 완전연결계층에 의한 변환은 `선형 변환` 이다.
- 여기에 `비선형` 효과를 부여하는 것이 바로 활성화 함수(activation function)이다. 
- 비선형 활성화 함수를 이용함으로써 신경망의 표현력을 높일 수 있다.
    - e.g. 시그모이드 함수

In [2]:
# 시그모이드 함수
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [5]:
a = sigmoid(h)
print(a)
print(a.shape)

[[0.02134535 0.80651937 0.95808245 0.28679929]
 [0.2193597  0.11079429 0.35739525 0.28699375]
 [0.1541049  0.28123596 0.6428382  0.26871678]
 [0.07447536 0.51067215 0.83695427 0.27327574]
 [0.82726412 0.30617594 0.59978484 0.14636189]
 [0.01959511 0.91467963 0.98326966 0.25968751]
 [0.03123831 0.78892792 0.9518894  0.27255499]
 [0.28669533 0.31123468 0.65864134 0.23064472]
 [0.05969091 0.91561968 0.98163498 0.21214767]
 [0.028339   0.36780513 0.75957937 0.34280332]]
(10, 4)


In [7]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

x = np.random.randn(10, 2) # 2차원 데이터 10개가 미니배치로 처리
W1 = np.random.randn(2, 4) 
b1 = np.random.randn(4)
W2 = np.random.randn(4, 3)
b2 = np.random.randn(3)

h = np.matmul(x, W1) + b1
a = sigmoid(h)
s = np.matmul(h, W2) + b2 # (10, 3): 10개의 데이터가 한꺼번에 처리되었고, 각 데이터가 3차원 데이터로 변환 되었다는 뜻

- 이 신경망은 3차원 데이터를 출력한다.
- 따라서 각 차원의 값을 이용해 3 클래스 분류를 할 수 있고
    - 이 경우, 출력된 3차원 벡터의 차원은 각 클래스에 대응하는 점수(score)가 된다.
    - 여기서 score는 확률이 되기 전의 값.
    - score가 높을 수록 뉴런에 해당하는 클래스의 확률도 높아진다. 
    - 이 점수를 softmax 함수에 입력하면 확률을 얻을 수 있다.

### 1.2.2 계층으로 클래스화 및 순전파 구현
- 완전 연결 계층에 의한 변환을 `Affine` 계층으로
- 시그모이드 함수에 의한 변환을 `Sigmoid`계층으로 구현
    - 각 계층은 파이썬 클래스로 구현하며, 기본 변환을 수행하는 메서드의 이름은 `forward()`

- 신경망 추론과정에서 하는 처리는 신경망의 순전파

- 계층 구현 규칙
    - 모든 계층은 `forward()`와 `backward()` 메서드를 가진다.
        - 각각 순전파와 역전파를 수행하는 메서드
    - 모든 계층은 인스턴스 변수인 `params`와 `grads`를 가진다.
        - `params`: 가중치와 편향같은 매개 변수를 담는 리스트
        - `grads`: `params`에 저장된 각 매개변수에 대응하여, 해당 매개변수의 기울기를 보관하는 리스트

In [1]:
import numpy as np

class Sigmoid:
    def __init__(self):
        self.params = []

    def forward(self, x):
        return 1 / (1 + np.exp(-x))
        

In [2]:
class Affine:
    def __init__(self, W, b): # Affine 계층은 초기화될 때, 가중치와 편향을 받는다. 
        self.params = [W, b]

    def forward(self, x):
        W, b = self.params
        out = np.matmul(x, W) + b
        return out

In [3]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size):
        I, H, O = input_size, hidden_size, output_size

        # 가중치의 편향 초기화
        W1 = np.random.randn(I, H)
        b1 = np.random.randn(H)
        W2 = np.random.randn(H, O)
        b2 = np.random.randn(O)

        # 계층 생성
        self.layers = [
            Affine(W1, b1),
            Sigmoid(),
            Affine(W2, b2)
        ]

        # 모든 가중치를 리스트에 모은다 (학습해야할 가중치 매개변수들)
        self.params = []
        for layer in self.layers:
            self.params += layer.params
        
    def predict(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x

In [5]:
# TwoLayerNet 신경망 추론
x = np.random.randn(10, 2)
model = TwoLayerNet(2, 4, 3) # (Input, Hidden, Output)
score = model.predict(x)
print(score)
print(score.shape)

[[ 0.80320577 -0.48047912 -2.65068976]
 [ 0.63408267 -0.42542014 -2.72631863]
 [-0.36840586 -0.05893432 -3.04191695]
 [ 0.99688147 -0.54534897 -2.51368653]
 [-0.59384413  0.06626326 -2.92460612]
 [-0.40408894 -0.06034213 -3.05564793]
 [-0.97035052  0.10932709 -2.95232129]
 [ 0.43996711 -0.33898053 -2.841006  ]
 [ 0.47972864 -0.35909694 -2.8280516 ]
 [ 0.97580683 -0.53611631 -2.48589194]]
(10, 3)
